# Kapitel 0: Einleitung und Überblick

# Kapitel 1: Riemann-Integrale mit Ober- und Untersummen, Trapezregel

# Kapitel 2: Monte-Carlo-Integration

# Kapitel 3: Visualisierung der Ergebnisse

# Kapitel 4: Testfälle und Evaluation

# Kapitel 5: Grenzen numerischer Integration

# Kapitel 6: Fakultativ: Gaußsche Fehlerfunktion

# Kapitel 7: Zusammenfassung und Fazit

## Anhang: Quellen, Anmerkungen, Gruppenmitglieder